У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [20]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, precision_score, recall_score
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [41]:
df = pd.read_csv('customer_segmentation_train.csv')
df = df.drop(columns=['ID'])
target = df['Segmentation']
features = df.drop(columns=['Segmentation'])
features_train, features_test, target_train,target_test = train_test_split(features, target, test_size=0.2, random_state=42,stratify=target )

In [81]:
df

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A
...,...,...,...,...,...,...,...,...,...,...
8063,Male,No,22,No,NaN,0.0,Low,7.0,Cat_1,D
8064,Male,No,35,No,Executive,3.0,Low,4.0,Cat_4,D
8065,Female,No,33,Yes,Healthcare,1.0,Low,1.0,Cat_6,D
8066,Female,No,27,Yes,Healthcare,1.0,Low,4.0,Cat_6,B


In [75]:
target_train.value_counts()

,count
Segmentation,
D,1814
A,1578
C,1576
B,1486


In [42]:
input_cols = list(features)
target_col = 'Segmentation'

train_inputs = features_train[input_cols].copy()
train_targets = target_train.copy()

test_inputs = features_test[input_cols].copy()
test_targets = target_test.copy()

numeric_cols = train_inputs.select_dtypes(include=np.number).columns.tolist()
categorical_cols = train_inputs.select_dtypes('object').columns.tolist()

In [57]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

X_train = preprocessor.fit_transform(train_inputs)
X_test = preprocessor.transform(test_inputs)

**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [72]:
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTENC
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

In [73]:
from imblearn.over_sampling import SMOTENC
from sklearn.svm import LinearSVC

smote = SMOTE(random_state=0)
X_train_smote, y_train_smote = smote.fit_resample(X_train, train_targets)

# Train linear SVC
clf_smote = SVC(kernel='linear',probability=True)
clf_smote.fit(X_train_smote, y_train_smote)

SVC(kernel='linear', probability=True)

In [74]:
y_train_smote.value_counts()

,count
Segmentation,
A,1814
B,1814
C,1814
D,1814


In [79]:
from imblearn.combine import SMOTETomek
from sklearn.svm import LinearSVC

smotetomek = SMOTETomek(random_state=0)
X_train_smotetomek, y_train_smotetomek = smotetomek.fit_resample(X_train, train_targets)

clf_smotetomek = SVC(kernel='linear',probability=True)
clf_smotetomek.fit(X_train_smotetomek, y_train_smotetomek)

SVC(kernel='linear', probability=True)

In [80]:
y_train_smotetomek.value_counts()

,count
Segmentation,
C,1460
D,1444
B,1389
A,1367


In [100]:
from imblearn.over_sampling import SMOTENC

preprocessor_basic = ColumnTransformer(transformers=[
    ('num', SimpleImputer(strategy='mean'), numeric_cols),
    ('cat', SimpleImputer(strategy='constant', fill_value='missing'), categorical_cols)
])

X_train_basic = preprocessor_basic.fit_transform(train_inputs)

all_features = numeric_cols + categorical_cols

cat_feature_indeces = [all_features.index(col) for col in categorical_cols]

smotenc = SMOTENC(categorical_features=cat_feature_indeces, random_state=42)
X_train_smotenc, y_train_smotenc = smotenc.fit_resample(X_train_basic, train_targets)


In [101]:
y_train_smotenc.value_counts()

,count
Segmentation,
A,1814
B,1814
C,1814
D,1814


**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [102]:
from sklearn.metrics import classification_report, precision_score, recall_score

In [103]:
log_reg = LogisticRegression(solver='liblinear')
ovr_model = OneVsRestClassifier(log_reg)
ovr_model.fit(X_train, train_targets)
ovr_predictions = ovr_model.predict(X_test)

print(classification_report(test_targets, ovr_predictions))

              precision    recall  f1-score   support

           A       0.42      0.46      0.44       394
           B       0.42      0.17      0.24       372
           C       0.49      0.63      0.55       394
           D       0.65      0.76      0.70       454

    accuracy                           0.52      1614
   macro avg       0.50      0.51      0.48      1614
weighted avg       0.50      0.52      0.50      1614



In [108]:
ovr_model.predict_proba(X_test)[:10].round(2)

array([[0.48, 0.19, 0.06, 0.27],
       [0.06, 0.05, 0.07, 0.82],
       [0.43, 0.3 , 0.13, 0.14],
       [0.43, 0.18, 0.02, 0.37],
       [0.04, 0.05, 0.13, 0.78],
       [0.51, 0.13, 0.01, 0.36],
       [0.44, 0.24, 0.12, 0.2 ],
       [0.27, 0.36, 0.35, 0.02],
       [0.08, 0.31, 0.61, 0.  ],
       [0.22, 0.32, 0.32, 0.14]])

In [104]:
log_reg = LogisticRegression(solver='liblinear')
ovr_model = OneVsRestClassifier(log_reg)
ovr_model.fit(X_train_smote, y_train_smote)
ovr_predictions = ovr_model.predict(X_test)

print(classification_report(test_targets, ovr_predictions))

              precision    recall  f1-score   support

           A       0.43      0.49      0.46       394
           B       0.39      0.25      0.30       372
           C       0.50      0.57      0.53       394
           D       0.68      0.71      0.70       454

    accuracy                           0.52      1614
   macro avg       0.50      0.51      0.50      1614
weighted avg       0.51      0.52      0.51      1614



In [109]:
ovr_model.predict_proba(X_train_smote)[:10].round(2)

array([[0.4 , 0.28, 0.2 , 0.12],
       [0.24, 0.29, 0.46, 0.02],
       [0.45, 0.21, 0.13, 0.22],
       [0.04, 0.26, 0.69, 0.01],
       [0.42, 0.14, 0.03, 0.4 ],
       [0.27, 0.35, 0.35, 0.04],
       [0.49, 0.26, 0.1 , 0.15],
       [0.34, 0.2 , 0.13, 0.33],
       [0.38, 0.29, 0.09, 0.24],
       [0.36, 0.3 , 0.09, 0.25]])

In [105]:
log_reg = LogisticRegression(solver='liblinear')
ovr_model = OneVsRestClassifier(log_reg)
ovr_model.fit(X_train_smotetomek, y_train_smotetomek)
ovr_predictions = ovr_model.predict(X_test)

print(classification_report(test_targets, ovr_predictions))

              precision    recall  f1-score   support

           A       0.42      0.49      0.46       394
           B       0.37      0.22      0.28       372
           C       0.50      0.58      0.53       394
           D       0.68      0.70      0.69       454

    accuracy                           0.51      1614
   macro avg       0.49      0.50      0.49      1614
weighted avg       0.50      0.51      0.50      1614



In [110]:
ovr_model.predict_proba(X_train_smotetomek)[:10].round(2)

array([[0.4 , 0.28, 0.2 , 0.12],
       [0.24, 0.29, 0.46, 0.02],
       [0.45, 0.21, 0.13, 0.22],
       [0.42, 0.14, 0.03, 0.4 ],
       [0.27, 0.35, 0.35, 0.04],
       [0.34, 0.2 , 0.13, 0.33],
       [0.38, 0.29, 0.09, 0.24],
       [0.36, 0.3 , 0.09, 0.25],
       [0.1 , 0.13, 0.15, 0.62],
       [0.51, 0.12, 0.01, 0.35]])

In [116]:
encoder_smotenc = ColumnTransformer(transformers=[
    ('num', MinMaxScaler(), list(range(len(numeric_cols)))),
    ('cat', OneHotEncoder(handle_unknown='ignore'), list(range(len(numeric_cols), len(all_features))))
])

ovr_model = Pipeline(steps=[
    ('prep', encoder_smotenc),
    ('clf', OneVsRestClassifier(LogisticRegression(solver='liblinear')))
])

ovr_model.fit(pd.DataFrame(X_train_smotenc), y_train_smotenc)

X_test_prep = pd.DataFrame(preprocessor_basic.transform(test_inputs))

ovr_predictions = ovr_smotenc.predict(X_test_prep)
print(classification_report(test_targets, ovr_predictions))

              precision    recall  f1-score   support

           A       0.42      0.48      0.45       394
           B       0.39      0.24      0.30       372
           C       0.51      0.58      0.54       394
           D       0.67      0.72      0.69       454

    accuracy                           0.52      1614
   macro avg       0.50      0.50      0.50      1614
weighted avg       0.51      0.52      0.51      1614



Для аналізу можна вибрати метрику macro avg. Найкращої моделі немає так я всі вони дають macro avg  на рівні 0,5.
Причиною цього можливо є те що модель не бінарна, а мультикласова, тобто в даній ситуації ми маємо 4 класи.

В усіх моделях precision для категорії є D є найбільшим можливо ця категорія має найбільш виражені ознаки, у порівнянні з іншими, інші можливо більше пересікаються
